In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import keras
import pandas
from enum import Enum
import json
import random
import cv2
from scipy.misc import toimage
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

# import utils

Using TensorFlow backend.


In [2]:
np.random.seed(100)
train_file = 'data/train/train.json'
test_file = 'data/test/test.json'
IMG_HEIGHT = 75
IMG_WIDTH = 75

In [37]:
def load_data_from_json(file_path, is_train_data=True):
    data = pd.read_json(file_path)
    data.inc_angle = data.inc_angle.replace('na', 0)
    data.inc_angle = data.inc_angle.astype(float).fillna(0.0)
    ids = data["id"]
    x_band1 = np.array([np.array(band).astype(np.float32).reshape(IMG_HEIGHT, IMG_WIDTH) for band in data["band_1"]])
    x_band2 = np.array([np.array(band).astype(np.float32).reshape(IMG_HEIGHT, IMG_WIDTH) for band in data["band_2"]])
    X = np.concatenate([x_band1[:, :, :, np.newaxis]
                              , x_band2[:, :, :, np.newaxis]
                             , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
    X_angle = np.array(data.inc_angle)
    if is_train_data:
        y = np.array(data["is_iceberg"])
    else:
        y = None
    return X, X_angle, y, ids

In [38]:
X_train, X_angle_train, y_train, _ = load_data_from_json(train_file)

In [6]:
# n_samples, nx, ny, nz = X_train.shape
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(np.reshape(X_train, (n_samples, nx*ny*nz)))
# X_train = np.reshape(X_train, (n_samples, nx, ny, nz))

In [8]:
train_generator = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.5,
    height_shift_range=0.5,    
    zoom_range=0.3,        
    horizontal_flip=True,
    vertical_flip=True,
)


In [23]:
def build_model():
    bn_model = 0.0
    p_activation = "relu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation)(input_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(256, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = BatchNormalization(momentum=0)(input_1)
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation)(input_1)
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
#     img_concat =  Concatenate()([img_1, img_2, input_2])
    
    dense_layer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_layer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_layer) ))
    
#     dense_layer = Dropout(0.5)(img_concat)
#     dense_layer = Dense(256, activation=p_activation)(dense_layer)
#     dense_layer = Dropout(0.5)(dense_layer)
#     dense_layer = Dense(64, activation=p_activation)(dense_layer)
    output = Dense(1, activation="sigmoid")(dense_layer)
    
    model = Model([input_1,input_2],  output)
    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = build_model()
    

In [24]:
def get_callbacks(filepath, patience=2):
#     es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]
    

In [ ]:
def data_generator(generator, angle, batch_size):
    img_generator = train_generator.flow(X_train, y_train, batch_size=batch_size, shuffle=False)
    data_len = len(angle)
    i = 0
    while True:
        images, labels = img_generator.next()
        angles = np.zeros(data_len, dtype=np.float32)
        if i + batch_size > data_len:
            end = data_len - i
            begin = (i+batch_size) - data_len
            angles[:end] = angle[i:data_len]
            angles[end:] = angle[:begin]
        else:
            angles = angle[i:i+batch_size]
        
        yield images, angles, labels
        
    
    

In [26]:
file_path = "model.h5"
callbacks = get_callbacks(filepath=file_path, patience=10)
batch_size=32

train_gen = train_generator.flow(X_train, y_train, batch_size=batch_size, shuffle=False)
# model.fit(x=[X_train, X_angle_train], y=y_train, batch_size=batch_size,
#                     validation_split=0.2,
#                     epochs=50,
#                     callbacks=callbacks
#                    )

model.fit_generator(generator=train_gen
)

# model.fit(X_train, y_train, epochs=25,
#         validation_split=0.15, batch_size=32
#          , callbacks=callbacks)

Train on 1283 samples, validate on 321 samples
Epoch 1/50
1283/1283 [==============================] - 3s - loss: 0.8660 - acc: 0.6025 - val_loss: 8.6711 - val_acc: 0.3832
Epoch 2/50
1283/1283 [==============================] - 2s - loss: 0.7533 - acc: 0.6594 - val_loss: 5.5233 - val_acc: 0.6573
Epoch 3/50
1283/1283 [==============================] - 2s - loss: 0.6844 - acc: 0.6726 - val_loss: 10.3460 - val_acc: 0.3489
Epoch 4/50
1283/1283 [==============================] - 2s - loss: 0.5861 - acc: 0.7202 - val_loss: 7.8971 - val_acc: 0.4829
Epoch 5/50
1283/1283 [==============================] - 2s - loss: 0.5260 - acc: 0.7685 - val_loss: 5.5233 - val_acc: 0.6573
Epoch 6/50
1283/1283 [==============================] - 2s - loss: 0.5211 - acc: 0.7685 - val_loss: 5.2512 - val_acc: 0.6573
Epoch 7/50
1283/1283 [==============================] - 2s - loss: 0.4971 - acc: 0.7786 - val_loss: 0.9537 - val_acc: 0.8037
Epoch 8/50
1283/1283 [==============================] - 2s - loss: 0.4437 - a

In [39]:
model.load_weights(filepath=file_path)
X_test, X_angle_test, _, test_ids = load_data_from_json(test_file, is_train_data=False)
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)


8400/8424 [============================>.] - ETA: 0s

In [40]:
submission = pd.DataFrame({'id': test_ids, 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.to_csv("submission.csv", index=False)
